In [37]:
import networkx as nx

from graph import PangenomeGraph
import numpy as np
from utils import _node_complement, read_gfa, _edge_complement

In [2]:
%%time
# gfa_file = "/Users/loconnor/Downloads/smaller_sub.gfa"
gfa_file = "/Users/loconnor/Downloads/grch38_chr20.gfa"
ref_walk = 7
# gfa_file = "/Users/loconnor/Dropbox/GitHub/graph_var/c4a.gfa"
# gfa_file = "test.gfa"
pg7, walks, _ = PangenomeGraph.from_gfa(gfa_file, reference_path_index=ref_walk, return_walks=True)

Num of binodes: 1859947
Num of biedges: 2574969
Computing reference tree
Computing positions
Computing branch points
CPU times: user 4min 32s, sys: 4.25 s, total: 4min 36s
Wall time: 4min 38s


In [40]:
%%time
# gfa_file = "/Users/loconnor/Downloads/smaller_sub.gfa"
# gfa_file = "/Users/loconnor/Downloads/grch38_chr20.gfa"
# ref_walk = 7
gfa_file = "/Users/loconnor/Dropbox/GitHub/graph_var/c4a.gfa"
# gfa_file = "test.gfa"
pg = PangenomeGraph.from_gfa(gfa_file, reference_path_index=ref_walk, return_walks=False)

Num of binodes: 8
Num of biedges: 13
Computing reference tree
Computing positions
Computing branch points
CPU times: user 2.26 ms, sys: 1.47 ms, total: 3.73 ms
Wall time: 4.11 ms


In [42]:
pg.write_tree('/Users/loconnor/Downloads/c4a.tree')

['+_terminus_+',
 'SKIC2_+',
 'DXO_-',
 'STK19_+',
 'C4A_+',
 'CYP21A2_+',
 'TNXB_-',
 'C4B_+',
 'ATF6B_-',
 '-_terminus_+']

In [43]:
pg_from_file = PangenomeGraph.from_gfa(gfa_file, reference_path_index=ref_walk, return_walks=False, edgeinfo_file='/Users/loconnor/Downloads/c4a.tree')

Num of binodes: 8
Num of biedges: 13
Reading edgeinfo file


IndexError: list index out of range

In [3]:
inversions = [e for e in pg7.variant_edges if pg7.is_inversion(e)]

In [4]:
len(inversions)

2

In [5]:
pg7.reference_tree.number_of_nodes() * 2

3719898

In [6]:
pg7.number_of_nodes()

3719898

In [7]:
pg7.out_degree('-_terminus_-'), pg7.in_degree('-_terminus_+'), pg7.out_degree('+_terminus_+'), pg7.in_degree('+_terminus_-')

(362, 362, 345, 345)

In [8]:
nx.is_weakly_connected(pg7)

True

In [9]:
nx.is_weakly_connected(pg7.reference_tree)

True

In [10]:
len(list(nx.weakly_connected_components(pg7)))

1

In [11]:
ref_path_dir = [pg7.nodes[node]['direction'] for node in pg7.reference_path]
np.all(np.array(ref_path_dir) == 1)

np.True_

In [12]:
inversions

[('42682860_+', '42682867_-'), ('42737146_-', '42737147_+')]

In [20]:
deletions = [e for e in pg7.variant_edges if pg7.is_forward_edge(e)]
len(deletions)

85441

In [24]:
def is_snp(G, edge):
    if not G.is_crossing_edge(edge):
        return False
    ref, alt, _, _ = G.ref_alt_alleles(edge)
    return len(ref) == len(alt) == 1
snps = [e for e in pg7.variant_edges if is_snp(pg7, e)]
len(snps)

521930

In [21]:
repeats = [e for e in pg7.variant_edges if pg7.is_back_edge(e)]
len(repeats)

4613

In [22]:
len(pg7.variant_edges)

715726

In [25]:
521930/715726

0.7292315774472354

In [30]:
allele_length = []
for n,e in enumerate(pg7.variant_edges):
    if n > 100:
        break
    ancestor = pg7.edges[e]['branch_point']
    descendent = e[0] if e[0] in pg7.reference_tree else _node_complement(e[0])
    allele_length.append(len(pg7.walk_up_tree(ancestor, descendent, search_limit=np.inf)))
    
    

StopIteration: 

In [38]:
def annotate_branch_points(self) -> None:
    """
    Computes the branch point, i.e. the lowest common ancestor in the reference tree, of each variant edge.
    """

    non_inversion_variants = [(u, v) for u, v in self.variant_edges
                              if self.nodes[u]['direction'] == self.nodes[v]['direction']]

    positive_direction_variants = [e if self.nodes[e[0]]['direction'] == 1 else _edge_complement(e)
                                   for e in non_inversion_variants]

    # For each variant edge (u,v), yield the lowest common ancestor of u and v in the tree
    branch_point_tuples = nx.tree_all_pairs_lowest_common_ancestor(
        self.reference_tree,
        root=self.termini[0]+'_+',
        pairs=positive_direction_variants
    )

    for edge, branch_point in branch_point_tuples:
        self.edges[edge]['branch_point'] = branch_point
        self.edges[_edge_complement(edge)]['branch_point'] = _node_complement(branch_point)

In [39]:
annotate_branch_points(pg7)

In [32]:
pg7.edges[e]['branch_point']

'42729902_-'

In [33]:
pg7.edges[e]['branch_point'] in pg7.reference_tree

False

In [34]:
_node_complement(pg7.edges[e]['branch_point']) in pg7.reference_tree

True

In [36]:
pg7.nodes[_node_complement(pg7.edges[e]['branch_point'])]['direction']

1